# RNN em Física: Análise de sentimento em texto - banco de dados de review de filmes no IMDB

## Alexandre Suaide

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from keras.preprocessing import text_dataset_from_directory
from keras.models import Sequential, load_model
from keras.layers import Dense, LSTM, Input, TextVectorization, Embedding
from keras.optimizers import Adam

from tensorflow.strings import regex_replace
from tensorflow import convert_to_tensor

from sklearn.preprocessing import MinMaxScaler

<h3>Lê os dados dos diretório</h3>
A estrutura do diretório deve ser, de acordo com a documentação do Keras:
<pre>
main_directory/
...class_a/
......a_text_1.txt
......a_text_2.txt
...class_b/
......b_text_1.txt
......b_text_2.txt
</pre>
Ver documentação <a href="https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text_dataset_from_directory"> aqui </a>

In [ ]:
def readData(dir):
  data = text_dataset_from_directory(dir)
  return data.map(
    lambda text, label: (regex_replace(text, '<br />', ' '), label), # os arquivos possuem quebra de linha
  )

Esses dados são baseados em review de filmes do site IMDB e formam um banco de dados disponível originalmente em http://ai.stanford.edu/~amaas/data/sentiment/

In [ ]:
datadir = "../data/imdb"
data_train = readData(datadir+"/train")
data_test  = readData(datadir+"/test")

text_train = data_train.map(lambda text, label: text)

<h3>Vamos ver uns dados</h3>

In [ ]:
for text, label in data_train.take(4):
    print(text.numpy()[0])
    print(label.numpy()[0])

<h3>Vamos criar a nossa rede</h3>

In [ ]:
model = Sequential()
model.add(Input(shape=(1,), dtype="string"))

<h3>Vamos criar uma camada de vetores de texto.</h3>h3>
Dois parâmetros importantes são o tamanho do vocabulário e a dimensionalidade do vetor. Antes de adicionar a camada ao modelo, precisamos adaptar essa camada ao textos que serão usados. No caso de estourar o tamanho máximo do vocabulários, novas palavras serão classificadas como "out of vocabulary" 00V

In [ ]:
max_tokens = 2000  # tamanho do vocabulário
max_dim = 300 # dimensionalidade do vetor. No fundo significa que apenas os max_dim primeiros tokens serão usados para converter um texto em números
vector_layer = TextVectorization(max_tokens=max_tokens, output_mode="int", output_sequence_length=max_dim)
vector_layer.adapt(text_train)
model.add(vector_layer)

Vamos ver as palavras mais frequêntes no dicionário

In [ ]:
vocabulary = vector_layer.get_vocabulary()
n = 20
for index in range(n):
    print(f'{index}: {vocabulary[index]} ')

Vamos vetorizar uma frase

In [ ]:
frase = [["But honestly, it doesn't even matter what the film is about."]]
vetor = vector_layer(frase)
vetor

<h3>Criar uma camada de embedding</h3> Note que o tamanho do vocabulário é max_tokens + 1 por conta do out of vocabulary

In [ ]:
embedding_layer = Embedding(input_dim = max_tokens + 1, output_dim = 128)
model.add(embedding_layer)

Vamos passar o vetor anterior pela camada de embedding e ver o que sai

In [ ]:
resultado = embedding_layer(vetor)
resultado

<h3>Vamos adicionar as camadas restantes, sendo uma delas do tipo LSTM</h3>

In [ ]:
model.add(LSTM(64))
model.add(Dense(64, activation="relu"))
model.add(Dense(1, activation="sigmoid"))

In [ ]:
optimizer = Adam()

In [ ]:
model.compile(optimizer=optimizer, loss='binary_crossentropy',  metrics=['accuracy'])

In [ ]:
model.fit(data_train, validation_data = data_test, epochs = 10)

<h3>Vamos ver como a camada de embedding ficou depois de ser treinada</h3>

In [ ]:
resultado = embedding_layer(vetor)
resultado

<h3>Salvando o modelo</h3>
Treinar um modelo de linguagem leva tempo e custa CPU/GPU. Normalmente você não faz isso o tempo todo. Você treina o seu modelo e salva as saídas. Vamos fazer isso e depois usar esse treinamento em uma outra rede

In [ ]:
file = 'meu_modelo.keras'
model.save(file)

<h3>Vamos criar um novo modelo e carregar esse treinamento para usar</h3>

In [ ]:
uso = load_model(file)

In [ ]:
meu_review = ["""I had already watched this movie, but I remembered almost nothing, including that several interesting actors are part of it. 
The opening scene is intensely gory and very interesting, with crude visual effects worthy of 2002, but very pleasing to the eyes of fans of B movies from the 90s/00s. 
Without a doubt, it is a film worth watching and rewatching, especially if the viewer enjoys supernatural exploitation films.
"""]

In [ ]:
print(uso.predict(convert_to_tensor(meu_review)))

In [ ]:
meu_review = ["""It has no plot, no comedy, no drama, no passion. You basically waste 2 hours of your time watching these characters you just can't seem to get attached to... 
Watch it only if you're interested in watching all these actors as they were before they got famous, or if you feel REALLY nostalgic about the '80s.
"""]

In [ ]:
print(uso.predict(convert_to_tensor(meu_review)))

In [ ]:
meu_review = ["""I usually steer toward Sci-Fi and Fantasy movies, but after watching the The Breakfast Club, I thought I may enjoy seeing actors and actresses like Judd Nelson, and Ally Sheedy working together again. 
Well, I was right. Some movies can make you feel different emotions by being tearful, or violent. St.Elmo's Fire made me cry- not because of the actual plot, but the way it truly played to real life. 
The aspects of this movie letf wishing to see what would happen next, after the character's old lives were left behind."""]

In [ ]:
print(uso.predict(convert_to_tensor(meu_review)))